In [1]:
import os
import re
import fitz  # PyMuPDF for PDF text extraction
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from pdf2image import convert_from_path
from pylatex import Document
from dotenv import load_dotenv
import subprocess

In [2]:
# Load API key from .env file
load_dotenv()
api_key_groq = os.getenv("GROQ_API_KEY")

In [3]:
# Define directories
DATA_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\Data"
OUTPUT_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output"


In [4]:

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text



In [5]:
prev_pdf_path = os.path.join(DATA_DIR, "prev_year_copy.pdf")
curr_pdf_path = os.path.join(DATA_DIR, "curr_year_report.pdf")

prev_text_path = os.path.join(DATA_DIR, "prev_text.txt")
curr_text_path = os.path.join(DATA_DIR, "curr_text.txt")

prev_year_text = extract_text_from_pdf(prev_pdf_path)
curr_year_text = extract_text_from_pdf(curr_pdf_path)

# Save for reference
with open(prev_text_path, "w", encoding="utf-8") as f:
    f.write(prev_year_text)

with open(curr_text_path, "w", encoding="utf-8") as f:
    f.write(curr_year_text)


In [6]:
import re

def replace_numbers_with_blanks(text):
    # Regex pattern to match numbers, decimals, percentages, and currency values
    pattern = r"\b\d+[\.,]?\d*\b|\b\d{1,3}(?:,\d{3})*\b|\$\d+[\.,]?\d*|\d+%"
    
    # Replace matched numbers with "__"
    blank_text = re.sub(pattern, "__", text)
    
    return blank_text

# Apply the function to remove numbers from previous year report
blank_prev_year_text = replace_numbers_with_blanks(prev_year_text)

blank_prev_year_text_path = os.path.join(OUTPUT_DIR, "blank_prev_year_text.txt")
# Save blank text (for debugging, optional)
with open(blank_prev_year_text_path, "w", encoding="utf-8") as f:
    f.write(blank_prev_year_text)

print("Numbers replaced with blanks successfully!")


Numbers replaced with blanks successfully!


In [10]:
import fitz  # PyMuPDF
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

prev_pdf_path = os.path.join(DATA_DIR, "prev_year_copy.pdf")  # Reference PDF
blank_text_path = os.path.join(OUTPUT_DIR, "blank_prev_year_text.txt")  # Text file with updated numbers
output_pdf_path = os.path.join(OUTPUT_DIR, "formatted_updated_report.pdf")  # Output PDF

# ✅ Step 1: Read the blank text file (with filled blanks)
with open(blank_text_path, "r", encoding="utf-8") as file:
    blank_text = file.read()  # Read the entire text

# ✅ Step 2: Extract original layout from `prev_year_copy.pdf`
doc = fitz.open(prev_pdf_path)
page_count = len(doc)

# ✅ Step 3: Create a new PDF while keeping the formatting similar to `prev_year_copy.pdf`
c = canvas.Canvas(output_pdf_path, pagesize=letter)
width, height = letter  # Get page size

line_height = height - 50  # Start text placement from top
left_margin = 50  # Set left margin
max_width = width - 100  # Set right margin

text_lines = blank_text.split("\n")  # Split text into lines

for line in text_lines:
    if line.strip():  # Avoid empty lines
        c.setFont("Helvetica", 10)  # Standard font to avoid missing font errors
        words = line.split()  # Split words for wrapping

        current_line = ""
        for word in words:
            # Check if the line is too long, wrap if necessary
            if c.stringWidth(current_line + " " + word, "Helvetica", 10) < max_width:
                current_line += " " + word
            else:
                c.drawString(left_margin, line_height, current_line.strip())  # Draw line
                line_height -= 15  # Move down
                current_line = word  # Start new line

        # Draw the last part of the line
        if current_line:
            c.drawString(left_margin, line_height, current_line.strip())
            line_height -= 15  # Move down

        # If we reach the bottom, start a new page
        if line_height < 50:
            c.showPage()
            line_height = height - 50

# ✅ Step 4: Save the final formatted PDF
c.save()
print("✅ Updated report saved successfully:", output_pdf_path)


✅ Updated report saved successfully: C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output\formatted_updated_report.pdf


In [15]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### STEP 3: SET UP LLM WITH GROQ ###
llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.2, api_key=api_key_groq)

prompt_template = PromptTemplate(
    input_variables=["prev_report", "curr_report"],
    template="""
    You are a professional document editor. Your task is to **ONLY** replace missing values (`____`) 
    in the previous year's report using the corresponding values from the current year's report, 
    while **keeping the document structure, wording, and formatting exactly the same**.

    **Rules:**
    - Do **not** modify any words, sentences, section titles, or formatting.
    - Only replace `____` placeholders with the correct values from the current report.
    - If a value does not exist in the current report, keep `____` unchanged.
    - Ensure that the response is **formatted exactly like the original document**.

    ---
    **Previous Year Report (with missing values):**
    {prev_report}

    ---
    **Current Year Report (reference for values):**
    {curr_report}

    ---
    **Updated Report (with values filled in):**
    """
)

### STEP 4: RUN LLM TO FILL BLANKS ###
llm_chain = LLMChain(llm=llm, prompt=prompt_template)
response = llm_chain.invoke({"prev_report": blank_prev_year_text, "curr_report": curr_year_text})


In [16]:
response['text']

'**Annual Financial Report - Adidas Ltd. 2024**\n\n**Executive Summary In 2024, Adidas Ltd. experienced a revenue increase of 9.8%, reaching a total of $24.7 billion. This was driven by digital transformation and sustainable product innovations. Net profit stood at $3.8 billion, reflecting a strong financial year. The company continued to expand its market presence in emerging economies while managing operating expenses of $15.1 billion.**\n\n**Financial Performance Adidas maintained a Debt-to-Equity Ratio of 1.6, ensuring financial stability despite inflationary pressures. Investments in direct-to-consumer sales channels helped drive a year-over-year growth of 2% in revenues. The company focused on optimizing production costs, which contributed to a steady profit margin.**\n\n**Market Trends & Outlook The sports apparel industry saw increasing demand for eco-friendly products, leading Adidas to invest in sustainable materials. In response to evolving consumer preferences, Adidas’s onl

In [17]:
# Define the output file path
output_txt_path = os.path.join(OUTPUT_DIR, "replaced.txt")

# Save the response to a text file
with open(output_txt_path, "w", encoding="utf-8") as file:
    file.write(response['text'])

print(f"✅ File saved successfully at: {output_txt_path}")


✅ File saved successfully at: C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output\replaced.txt


In [10]:
import fitz  # PyMuPDF
import os

OUTPUT_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output"

prev_pdf_path = os.path.join(DATA_DIR, "prev_year_copy.pdf")  # Original structured PDF
updated_text_path = os.path.join(OUTPUT_DIR, "replaced.txt")  # Text file with filled values
output_pdf_path = os.path.join(OUTPUT_DIR, "final.pdf")  # Final output PDF

# ✅ Step 1: Read the entire updated text file
with open(updated_text_path, "r", encoding="utf-8") as file:
    updated_text = file.read()

# ✅ Step 2: Open the original structured PDF to copy its format
prev_doc = fitz.open(prev_pdf_path)
new_doc = fitz.open()  # Create a new PDF

# ✅ Step 3: Extract the layout from the previous PDF
for page in prev_doc:
    new_page = new_doc.new_page(width=page.rect.width, height=page.rect.height)  # Copy page size

    # 🔹 Define text insertion position (Adjust if needed)
    text_x, text_y = 50, 50  # Starting position (Left margin, Top margin)
    
    # 🔹 Define font and size
    font_size = 11
    font_name = "helv"  # Default PDF font

    # ✅ Step 4: Insert text into the new PDF page
    new_page.insert_textbox(
        fitz.Rect(text_x, text_y, page.rect.width - 50, page.rect.height - 50),
        updated_text,
        fontsize=font_size,
        fontname=font_name,
        color=(0, 0, 0),
        align=0,  # Left-aligned text
    )

# ✅ Step 5: Save the final PDF
new_doc.save(output_pdf_path)
print("✅ Final structured PDF saved successfully:", output_pdf_path)


✅ Final structured PDF saved successfully: C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\output\final.pdf
